##### Copyright 2023 Brian Yarbrough.

# Tensorflow Lite

Previously, we trained a model based on MobileNetV2 to differentiate between cats and dogs.

Prior to conducting inference with this model, we will convert the model to Tensorflow Lite.
This will have performance advantages on constrained hardware.

## Upload the Model

First, upload the saved model (`cat-dog-tuned.zip` if you used the previous tutorial).
Then unzip the model.


In [ ]:
# Run after uploading the file
!unzip cat-dog-tuned.zip

## Convert the saved model

We'll use [TFLiteConverter](https://www.tensorflow.org/lite/api_docs/python/tf/lite/TFLiteConverter) to export the model to a single binary.

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(help(tf.lite.TFLiteConverter))

To convert the model we will open it and then follow [the docs](https://www.tensorflow.org/lite/models/convert/convert_models#convert_a_savedmodel_recommended_).

In [ ]:
saved_model_dir = "cat-dog-tuned"  # path to the SavedModel directory
tflite_model = "cat-dog.tflite" # what to save the converted model as

# Open the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# Use Dynamic Range Quantization
# https://www.tensorflow.org/lite/performance/post_training_quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Convert the model.
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

## Conduct inference on novel images

Now that we've converted the model, let's test it with novel images!
We can do this both with the full Keras API, or rely on the Tensorflow Lite runtime - based on our hardware and OS.

#### Upload images

You should see 7 `.jpg` images in the folder. Upload those to Colab prior to continuing, or feel free to grab your own images of cats and dogs!


### Use TF Keras API

This requires the full tensorflow install. It uses the original model saved in a directory.

In [ ]:
#use Keras API
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model('cat-dog-tuned')

for p in ['Vin.jpg', 'cat1.jpg', 'cat2.jpg', 'cat3.jpg', 'dog1.jpg', 'dog2.jpg', 'dog3.jpg']:
    '''Labels
        0 = Cat
        1 = Dog    
    '''
    # Load and resize the image
    img = tf.keras.utils.load_img(
        p, target_size=(160, 160)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch of size 1

    # Conduct inference and extract the result from the np array
    prediction = model.predict(img_array)
    result = np.squeeze(prediction)
    
    # This is an exponential function used to gauge confidence
    sig_result = tf.nn.sigmoid(result)
    sig_predict = tf.where(sig_result < 0.5, 0, 1)
    sig_predict = sig_predict.numpy()

    print("Pic:", p)
    print("prediction", result)
    print("Inferred label:", sig_predict)


### Use TF Lite Interpreter

This more closely mirrors what we'll do on our embedded system.
The only difference is we will use the included `tf.lite` module instead of the standalone `tflite-runtime`.

In [ ]:
# Use tf.lite interpreter
import tensorflow as tf # on embedded device use: import tflite_runtime.interpreter as tflite
import numpy as np
from PIL import Image

model_path = "cat-dog.tflite"

# For running on tflite-runtime replace this with tflite.Interpreter
interpreter = tf.lite.Interpreter(model_path=model_path)

# Embedded devices are memory constrained, so this handles that
interpreter.allocate_tensors()
# Details about model inputs and outputs
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]["shape"]

for p in ['Vin.jpg', 'cat1.jpg', 'cat2.jpg', 'cat3.jpg', 'dog1.jpg', 'dog2.jpg', 'dog3.jpg']:
    '''Labels
        0 = Cat
        1 = Dog    
    '''
    # Load the image using PIL
    image = Image.open(p)
    # Resize the image to match what the model was trained on
    resized_image = image.resize((input_shape[1], input_shape[2]))
    input_data = np.array(resized_image, dtype=np.float32)
    input_data = np.expand_dims(input_data, axis=0) # Create a batch of size 1

    # Conduct inference
    interpreter.set_tensor(input_details[0]["index"], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])
    # Pull out the raw value from the np array
    prediction = np.squeeze(output_data)

    # Computing exponents for sigmoid function is expensive, so use a simple heuristic instead.
    # If  you need an "unknown" option or confidence threshold, use something like this.
    # label = 0 if prediction < -3 else (1 if prediction > 3 else -1)
    label = 0 if prediction < 0 else 1

    print("Pic:", p)
    print("Prediction:", prediction)
    print("Inferred label:", label)


## Next step: embedded device!

Now go put the tflite model on an embedded device, such as Raspberry Pi or Arduino and conduct inference!

### Running on Raspberry Pi OS Bullseye 11

You can get started with:

```bash
pip install tflite-runtime==2.11.0
```

Then change the above code to use `tflite` instead of `tf.lite`, as annotated in the comments.

Finally, run it! Then considering using your picamera for live inference!